# Dataset
1. [Edmonton Property Assessment Data](https://data.edmonton.ca/City-Administration/Property-Assessment-Data-Current-Calendar-Year-/q7d6-ambg/data)
2. [EPS Neighbourhood Criminal Occurrences](https://dashboard.edmonton.ca/dataset/EPS-Neighbourhood-Criminal-Occurrences/xthe-mnvi/data)
3. [Edmonton Neighbourhood Boundaries](https://data.edmonton.ca/dataset/Neighbourhood-Boundaries-2019/nb64-h7uj)

In [20]:
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import pickle as pkl

In [29]:
file_name = "Property_Assessment_Data__Current_Calendar_Year_.csv"
df_prop = pd.read_csv(file_name)
df_prop.info()

# print(df_prop.iloc[:,6].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416090 entries, 0 to 416089
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Account Number        416090 non-null  int64  
 1   Suite                 81496 non-null   object 
 2   House Number          371650 non-null  float64
 3   Street Name           371650 non-null  object 
 4   Garage                416090 non-null  object 
 5   Neighbourhood ID      416084 non-null  float64
 6   Neighbourhood         416084 non-null  object 
 7   Ward                  416084 non-null  object 
 8   Assessed Value        416090 non-null  int64  
 9   Latitude              416090 non-null  float64
 10  Longitude             416090 non-null  float64
 11  Point Location        416090 non-null  object 
 12  Assessment Class % 1  416090 non-null  int64  
 13  Assessment Class % 2  1092 non-null    float64
 14  Assessment Class % 3  297 non-null     float64
 15  

In [50]:
# cleanup data
# Keep residential only
# df_prop['Assessment Class 1'].unique()

df_prop = df_prop[df_prop['Assessment Class 1'] == 'RESIDENTIAL']
df_prop = df_prop[df_prop["Garage"] == 'Y']
# print(df_prop.head(10))
# df_prop.info()

df2 = df_prop.filter(['Neighbourhood','Assessed Value','Latitude','Longitude'], axis=1)
df2.columns = ['neighbourhood','value','latitude','longitude']

# AVG value
df1 = df2.groupby(['neighbourhood'],as_index=False).mean()
df1.sort_values(by = ['value'], ascending = False, inplace = True)
df1.to_pickle("prop_avg_value.pkl")
df1


,neighbourhood,value,latitude,longitude
240,RIVER VALLEY CAMERON,1.987500e+06,53.479147,-113.640700
241,RIVER VALLEY FORT EDMONTON,1.983000e+06,53.497074,-113.587297
187,MATTSON,1.784000e+06,53.398244,-113.438253
31,BLACKMUD CREEK RAVINE,1.378500e+06,53.444808,-113.514817
243,RIVER VALLEY WINDERMERE,1.328750e+06,53.448221,-113.624327
...,...,...,...,...
151,KENNEDALE INDUSTRIAL,1.861667e+05,53.587971,-113.424088
143,INDUSTRIAL HEIGHTS,1.155000e+05,53.578939,-113.444506
303,WESTVIEW VILLAGE,9.076364e+04,53.552228,-113.696801
183,MAPLE RIDGE,8.996842e+04,53.501551,-113.363377


In [53]:
file_name = "EPS_Neighbourhood_Criminal_Occurrences.csv"
df_eps = pd.read_csv(file_name)
df_eps.head(5)

df = df_eps[df_eps["Occurrence Reported Year"] == 2019]
# print(df.iloc[:,0].unique())
df.columns = ["neighbourhood","type","year","quarter","month","occurrences"]
df.head(5)


# Remove the crime data of non-residential area
df = df[~df['neighbourhood'].str.contains('INDUSTRIAL')]
df = df[~df['neighbourhood'].str.contains('Not Entered')]
df = df[~df['neighbourhood'].str.contains('BUSINESS PARK')]

df2 = df.groupby(['neighbourhood'], as_index=False)['occurrences'].sum()
df2 = df2.sort_values(by = 'occurrences')
df2.to_pickle("eps.pkl")

df2

,neighbourhood,occurrences
135,HERITAGE VALLEY AREA,1
261,RURAL WEST BIG LAKE,1
37,BLACKMUD CREEK RAVINE,1
246,RIVER VALLEY TERWILLEGAR,1
260,RURAL SOUTH EAST,1
...,...,...
41,BOYLE STREET,434
186,MCCAULEY,542
208,OLIVER,545
66,CENTRAL MCDOUGALL,564


In [55]:
# merge two datasets
edm = pd.merge(df1, df2, on = 'neighbourhood' )
edm.info()

edm.head(10)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 0 to 279
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   neighbourhood  280 non-null    object 
 1   value          280 non-null    float64
 2   latitude       280 non-null    float64
 3   longitude      280 non-null    float64
 4   occurrences    280 non-null    int64  
dtypes: float64(3), int64(1), object(1)
memory usage: 13.1+ KB


,neighbourhood,value,latitude,longitude,occurrences
0,BLACKMUD CREEK RAVINE,1.378500e+06,53.444808,-113.514817,1
1,RIVER VALLEY WINDERMERE,1.328750e+06,53.448221,-113.624327,3
2,WINDSOR PARK,1.318071e+06,53.524861,-113.535037,30
3,MILL CREEK RAVINE NORTH,1.235667e+06,53.523450,-113.475627,6
4,WESTBROOK ESTATES,1.205298e+06,53.469773,-113.548681,23
5,HERITAGE VALLEY TOWN CENTRE AREA,1.191500e+06,53.410022,-113.542598,12
6,HAYS RIDGE AREA,9.636686e+05,53.417073,-113.570856,1
7,GRANDVIEW HEIGHTS,9.527887e+05,53.500396,-113.549065,6
8,RIVERVIEW AREA,9.474328e+05,53.428497,-113.670739,11
9,CRESTWOOD,9.432858e+05,53.534780,-113.570079,44


In [62]:
# edmonton boundaries

file_name = "Neighbourhood_Boundaries___2019.csv"
df3 = pd.read_csv(file_name)
df3.head(5)
df3.columns = ['id','neighbourhood','desc','geom']

df3['neighbourhood'] = df3['neighbourhood'].str.upper()

df3.to_pickle('edm_neighbourhood_boundaries.pkl')
df3


,id,neighbourhood,desc,geom
0,2260,EVANSDALE,"Evansdale is named for H.M.E. Evans, Mayor of ...",MULTIPOLYGON (((-113.47970029870794 53.6152271...
1,2580,NORTHMOUNT,Residential land accounts for almost 70 percen...,MULTIPOLYGON (((-113.48510468465751 53.6063060...
2,1170,PRINCE RUPERT,Prince Rupert lies within the old Hudson Bay C...,MULTIPOLYGON (((-113.51740826792107 53.5646264...
3,1230,SPRUCE AVENUE,Spruce Avenue's central location has attracted...,MULTIPOLYGON (((-113.51230654891363 53.5624829...
4,1250,WESTWOOD,The character of Westwood has changed consider...,MULTIPOLYGON (((-113.4922573373992 53.57661667...
...,...,...,...,...
394,4570,WEDGEWOOD HEIGHTS,Most of the roads in Wedgewood Heights are nam...,MULTIPOLYGON (((-113.6513913938477 53.47496340...
395,6070,CLOVERDALE,Cloverdale is situated along the south bank of...,MULTIPOLYGON (((-113.4715473884504 53.53286797...
396,6620,RIVER VALLEY GOLD BAR,This non-residential neighbourhood forms part ...,MULTIPOLYGON (((-113.43880458979793 53.5536047...
397,6180,EASTGATE BUSINESS PARK,This non-residential neighbourhood is largely ...,MULTIPOLYGON (((-113.39408831400553 53.5266634...
